In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("C:/Users/Student/Desktop/專題code/極端值處理及常態分佈圖/eng_schema_labelencoder_for_ML_v2.csv")

In [ ]:
df

,Unnamed: 0,quater,main building area,transaction date,transaction obj,building state,hall,room,bathroom,partition,...,lng,MRT_1km,MRT_500m,MRT_300m,park_500m,park_300m,elite_school,land shifting area_ping,building shifting area_ping,berth shifting area_ping
0,0,1,49.09,2012-10-12,1,0,2,2,1,1,...,121.562827,0,0,0,1,1,0,54.214876,306.479339,57.685950
1,1,1,113.67,2012-10-09,0,4,2,4,2,1,...,121.578889,0,0,0,1,0,0,174.082645,541.719008,0.000000
2,2,1,34.02,2012-10-20,1,4,1,2,2,1,...,121.549945,1,0,0,1,1,0,47.140496,206.115702,0.000000
3,3,1,27.68,2012-10-01,0,3,1,1,1,1,...,121.537224,1,1,1,1,1,1,35.305785,148.892562,0.000000
4,4,1,68.17,2012-10-08,1,4,2,3,2,1,...,121.542196,1,1,0,1,1,1,102.148760,499.900826,33.057851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526441,526441,2,55.45,2021-01-18,1,0,2,3,1,1,...,121.504100,1,1,0,1,1,0,34.148760,418.578512,100.495868
526442,526442,2,72.17,2016-08-27,1,0,2,3,2,1,...,121.504100,1,1,0,1,1,0,43.801653,525.884298,100.495868
526443,526443,2,58.84,2021-02-22,1,0,2,3,2,1,...,121.504100,1,1,0,1,1,0,36.165289,445.388430,100.495868
526444,526444,2,113.03,2021-01-04,1,0,2,4,2,1,...,121.504445,1,0,0,1,1,0,68.925620,765.487603,100.495868


In [ ]:
df.columns

Index(['Unnamed: 0', 'quater', 'main building area', 'transaction date',
       'transaction obj', 'building state', 'hall', 'room', 'bathroom',
       'partition', 'mgmt org', 'sale price', 'berth total price', 'district',
       'auxiliary building area', 'balcony area', 'elevator',
       'building completion year', 'property age', 'unit price_ping', 'city',
       'lat', 'lng', 'MRT_1km', 'MRT_500m', 'MRT_300m', 'park_500m',
       'park_300m', 'elite_school', 'land shifting area_ping',
       'building shifting area_ping', 'berth shifting area_ping'],
      dtype='object')

In [ ]:
# 轉成datetime格式
df['transaction date'] = pd.to_datetime(df['transaction date'])

In [ ]:
# 取年份
df['transaction year'] = df['transaction date'].dt.year
df['transaction year']

0         2012
1         2012
2         2012
3         2012
4         2012
          ... 
526441    2021
526442    2016
526443    2021
526444    2021
526445    2020
Name: transaction year, Length: 526446, dtype: int64

In [ ]:
df['transaction year'].value_counts(sort=True)

2013    87489
2020    64969
2014    64167
2019    58651
2018    54975
2017    53455
2015    50644
2016    45913
2012    23017
2021    19998
2011     1442
2010     1388
2009      291
2008       24
2007       14
2006        2
1975        1
2005        1
2001        1
1998        1
1992        1
1989        1
1974        1
Name: transaction year, dtype: int64

In [ ]:
# 移除 2012 年以前的資料筆數
# 剩餘 523278 筆
df = df[df['transaction year']>2011]

In [ ]:
# 確認行政區為41個
df['district'].unique()

array([18, 15, 34,  5,  8,  4, 11, 21,  7, 10, 14, 16, 12, 22, 27, 20,  6,
       23, 19, 38, 26, 35, 28,  9,  1,  2, 36, 25,  3, 40,  0, 24, 29, 31,
       32, 33, 39, 37, 13, 17, 30], dtype=int64)

In [ ]:
# 確認留下近2012年以後的資料
df['transaction year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
      dtype=int64)

In [ ]:
df_final = pd.DataFrame()

for i in df['transaction year'].unique():
    for j in df['district'].unique():
            
        # 篩選年份及行政區存成df1
        df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
        
        if not df1.empty:

            # 四分位距法移除離群值, 存成df2
            n=1.5
            #IQR = Q3-Q1
            IQR = np.percentile(df1['unit price_ping'],75) - np.percentile(df1['unit price_ping'],25)

            # outlier > Q3 + n*IQR , outlier < Q1 - n*IQR 
            df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]

            # 每批 df1 處理完後合併進 df_final
            df_final = df_final.append(df1)
        
        else:
            print(i,j)
        

<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 

<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


2012 32
2012 37
2012 13
2012 17
2012 30


<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec6

<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


...

<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 

2021 32


<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1=df1[df1['unit price_ping'] < np.percentile(df1['unit price_ping'],75)+n*IQR][df1['unit price_ping'] > np.percentile(df1['unit price_ping'],25)-n*IQR]


2021 13
2021 17
2021 30


<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]
<ipython-input-11-bc406f5ec616>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[ df['transaction year'] == i ][ df['district'] == j ]


In [ ]:
df_final

,Unnamed: 0,quater,main building area,transaction date,transaction obj,building state,hall,room,bathroom,partition,...,MRT_1km,MRT_500m,MRT_300m,park_500m,park_300m,elite_school,land shifting area_ping,building shifting area_ping,berth shifting area_ping,transaction year
0,0,1,49.09,2012-10-12,1,0,2,2,1,1,...,0,0,0,1,1,0,54.214876,306.479339,57.685950,2012
1,1,1,113.67,2012-10-09,0,4,2,4,2,1,...,0,0,0,1,0,0,174.082645,541.719008,0.000000,2012
2,2,1,34.02,2012-10-20,1,4,1,2,2,1,...,1,0,0,1,1,0,47.140496,206.115702,0.000000,2012
3,3,1,27.68,2012-10-01,0,3,1,1,1,1,...,1,1,1,1,1,1,35.305785,148.892562,0.000000,2012
4,4,1,68.17,2012-10-08,1,4,2,3,2,1,...,1,1,0,1,1,1,102.148760,499.900826,33.057851,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503070,503070,1,98.10,2021-01-08,0,1,0,0,0,0,...,0,0,0,0,0,0,218.214876,324.297521,0.000000,2021
520354,520354,2,77.48,2021-04-20,0,1,1,2,0,1,...,0,0,0,1,1,0,137.785124,290.049587,0.000000,2021
500901,500901,1,72.37,2021-01-15,0,4,2,3,2,1,...,0,0,0,1,0,0,76.595041,364.892562,0.000000,2021
517041,517041,2,97.12,2021-04-04,0,1,2,3,2,1,...,0,0,0,1,1,0,78.148760,352.760331,0.000000,2021


In [ ]:
df_final.reset_index(drop=True)

,Unnamed: 0,quater,main building area,transaction date,transaction obj,building state,hall,room,bathroom,partition,...,MRT_1km,MRT_500m,MRT_300m,park_500m,park_300m,elite_school,land shifting area_ping,building shifting area_ping,berth shifting area_ping,transaction year
0,0,1,49.09,2012-10-12,1,0,2,2,1,1,...,0,0,0,1,1,0,54.214876,306.479339,57.685950,2012
1,1,1,113.67,2012-10-09,0,4,2,4,2,1,...,0,0,0,1,0,0,174.082645,541.719008,0.000000,2012
2,2,1,34.02,2012-10-20,1,4,1,2,2,1,...,1,0,0,1,1,0,47.140496,206.115702,0.000000,2012
3,3,1,27.68,2012-10-01,0,3,1,1,1,1,...,1,1,1,1,1,1,35.305785,148.892562,0.000000,2012
4,4,1,68.17,2012-10-08,1,4,2,3,2,1,...,1,1,0,1,1,1,102.148760,499.900826,33.057851,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508426,503070,1,98.10,2021-01-08,0,1,0,0,0,0,...,0,0,0,0,0,0,218.214876,324.297521,0.000000,2021
508427,520354,2,77.48,2021-04-20,0,1,1,2,0,1,...,0,0,0,1,1,0,137.785124,290.049587,0.000000,2021
508428,500901,1,72.37,2021-01-15,0,4,2,3,2,1,...,0,0,0,1,0,0,76.595041,364.892562,0.000000,2021
508429,517041,2,97.12,2021-04-04,0,1,2,3,2,1,...,0,0,0,1,1,0,78.148760,352.760331,0.000000,2021


In [ ]:
df_final = df_final.reset_index(drop=True)

In [ ]:
file = "C:/Users/Student/Desktop/專題code/極端值處理及常態分佈圖/ML data_outliers deletion_IQR.csv"
df_final.to_csv(file, encoding="utf-8", index=False)